In [20]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.transforms import ToTensor, Grayscale
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class CNA(nn.Module):
    def __init__(self, in_nc, out_nc, stride=1):
        super().__init__()
        self.conv = nn.Conv2d(in_nc, out_nc, 3, stride=stride, padding=1, bias=False)
        self.norm = nn.BatchNorm2d(out_nc)
        self.act = nn.GELU()

    def forward(self, x):
        out = self.conv(x)
        out = self.norm(out)
        out = self.act(out)
        return out

class UnetBlock(nn.Module):
    def __init__(self, in_nc, inner_nc, out_nc, inner_block=None):
        super().__init__()
        self.conv1 = CNA(in_nc, inner_nc, stride=2)
        self.conv2 = CNA(inner_nc, inner_nc)
        self.inner_block = inner_block
        self.conv3 = CNA(inner_nc, inner_nc)
        self.conv_cat = nn.Conv2d(inner_nc + in_nc, out_nc, 3, padding=1)

    def forward(self, x):
        _, _, h, w = x.shape
        inner = self.conv1(x)
        inner = self.conv2(inner)
        print(inner.shape)
        if self.inner_block is not None:
            inner = self.inner_block(inner)
        inner = self.conv3(inner)
        inner = F.interpolate(inner, size=(h, w), mode='bilinear', align_corners=False)
        inner = torch.cat((x, inner), dim=1)
        out = self.conv_cat(inner)
        return out

class Unet(nn.Module):
    def __init__(self, nc, num_downs=6):
        super().__init__()
        self.cna1 = CNA(1, nc)
        self.cna2 = CNA(nc, nc)
        unet_block = None
        for i in range(num_downs - 3):
            unet_block = UnetBlock(8 * nc, 8 * nc, 8 * nc, unet_block)
        unet_block = UnetBlock(4 * nc, 8 * nc, 4 * nc, unet_block)
        unet_block = UnetBlock(2 * nc, 4 * nc, 2 * nc, unet_block)
        self.unet_block = UnetBlock(nc, 2 * nc, nc, unet_block)
        self.cna3 = CNA(nc, nc)
        self.conv_last = nn.Conv2d(nc, 1, 3, padding=1)

    def forward(self, x):
        out = self.cna1(x)
        out = self.cna2(out)
        out = self.unet_block(out)
        out = self.cna3(out)
        out = self.conv_last(out)
        return out

# Путь к папкам с изображениями и масками
image_folder = 'D:/pictures/'
mask_folder = 'D:/masks/'

# Создание экземпляра модели U-Net
unet_model = Unet(32, 3)

# Создание экземпляра класса Dataset
class CustomDataset(Dataset):
    def __init__(self, image_folder, mask_folder, transform=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.image_filenames = os.listdir(image_folder)
        self.mask_filenames = os.listdir(mask_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_folder, self.image_filenames[idx])
        mask_filename = self.image_filenames[idx].split('.')[0] + '_mask.png'
        mask_path = os.path.join(self.mask_folder, mask_filename)
        image = Image.open(image_path).convert('RGB')
        mask = Image.open(mask_path).convert('L')
        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)
        return image, mask

transform = transforms.Compose([transforms.ToTensor(), transforms.Grayscale()])

dataset = CustomDataset(image_folder, mask_folder, transform=transform)

# Создание DataLoader для загрузки данных в модель
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Проход по данным и применение модели
for images, masks in dataloader:
    result = unet_model(images)
    print(result.shape)


torch.Size([1, 64, 240, 256])
torch.Size([1, 128, 120, 128])
torch.Size([1, 256, 60, 64])
torch.Size([1, 1, 480, 512])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 1, 316, 320])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 1, 316, 320])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 1, 316, 320])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 1, 316, 320])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 1, 316, 320])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 1, 316, 320])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 1, 316, 320])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
tor

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

# Проверка доступности CUDA
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    raise Exception("CUDA недоступна. Проверьте наличие и правильность установки драйверов и CUDA.")

# Определение функции потерь
criterion = nn.CrossEntropyLoss()

# Определение оптимизатора
learning_rate = 0.001
optimizer = optim.Adam(unet_model.parameters(), lr=learning_rate)

# Определение функции для обучения модели
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    model.to(device)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)
            
            # Обнуление градиентов
            optimizer.zero_grad()
            
            # Прямой проход
            outputs = model(images)
            
            # Вычисление функции потерь
            loss = criterion(outputs, masks)
            
            # Обратное распространение и оптимизация
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")
    
    print("Обучение завершено.")

# Параметры обучения
num_epochs = 10
batch_size = 1

# Создание DataLoader для обучения
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Обучение модели
train_model(unet_model, train_dataloader, criterion, optimizer, num_epochs)

# Применение обученной модели
# ...
# Код для применения модели к новым изображениям


Exception: CUDA недоступна. Проверьте наличие и правильность установки драйверов и CUDA.

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image

# Определение функции потерь
criterion = nn.CrossEntropyLoss()

# Определение оптимизатора
learning_rate = 0.001
optimizer = optim.Adam(unet_model.parameters(), lr=learning_rate)

# Определение функции для обучения модели
def train_model(model, dataloader, criterion, optimizer, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        for images, masks in dataloader:
            images = images.to(device)
            masks = masks.to(device)
            
            # Обнуление градиентов
            optimizer.zero_grad()
            
            # Прямой проход
            outputs = model(images)
            
            # Вычисление функции потерь
            loss = criterion(outputs, masks)
            
            # Обратное распространение и оптимизация
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss}")
    
    print("Обучение завершено.")

# Параметры обучения
num_epochs = 10
batch_size = 1

# Создание DataLoader для обучения
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Обучение модели
train_model(unet_model, train_dataloader, criterion, optimizer, num_epochs)

# Применение обученной модели
# ...
# Код для применения модели к новым изображениям


torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 240, 256])
torch.Size([1, 128, 120, 128])
torch.Size([1, 256, 60, 64])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 158, 160])
torch.Size([1, 128, 79, 80])
torch.Size([1, 256, 40, 40])
torch.Size([1, 64, 240, 256])
torch.Size([1, 128, 120, 128])
torch.Size([1, 256, 60, 64])
torch.Size([1, 64, 240, 256])
torch.Size([1, 128, 120, 128])
torch.Size([1, 256, 60, 64])
torch.Size([1, 64, 160, 16

KeyboardInterrupt: 